In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### Step 1. Import dependencies & download the dataset
We use `kagglehub` to pull the dataset *Sales Data for Economic Data Analysis* directly into our environment.


In [2]:
import os,glob,re
import pandas as pd
import numpy as np

import kagglehub

# Download latest version
path = kagglehub.dataset_download("abhishekrp1517/sales-data-for-economic-data-analysis")

print("Path to dataset files:", path)

all_files=[]
for ext in("*.csv","*.xlsx","*.xls","*.parquet"):
    all_files.extend(glob.glob(os.path.join(path,ext)))

print("files discovered:",all_files)

Path to dataset files: /kaggle/input/sales-data-for-economic-data-analysis
files discovered: ['/kaggle/input/sales-data-for-economic-data-analysis/salesforcourse-4fe2kehu.csv', '/kaggle/input/sales-data-for-economic-data-analysis/salesforcourse-4fe2kehu.xlsx']


### Step 2. Load the dataset
Select the best available file (CSV preferred). Then preview its shape and first rows.


In [7]:
import os, zipfile
from pathlib import Path

root = path  # from kagglehub download

def list_all_files(start_dir):
    out = []
    for r, _, files in os.walk(start_dir):
        for f in files:
            out.append(os.path.join(r, f))
    return out

files = list_all_files(root)
print("Found files initially:")
for f in files:
    print(" -", f)

# Unzip if .zip files are found
for f in files:
    if f.lower().endswith(".zip"):
        print("Unzipping:", f)
        with zipfile.ZipFile(f) as z:
            z.extractall(Path(f).parent)

# Re-list after unzip
files = list_all_files(root)
print("\nFiles after unzip:")
for f in files:
    print(" -", f)
from os.path import getsize

CAND_EXTS = (".csv", ".csv.gz", ".tsv", ".xlsx", ".xls", ".parquet", ".json", ".txt")

candidates = [f for f in files if f.lower().endswith(CAND_EXTS)]
if not candidates:
    raise FileNotFoundError("No candidate data files found (csv/xlsx/parquet/tsv/json/txt).")

# Sort by extension priority and file size
ext_priority = {".csv":1, ".csv.gz":1, ".tsv":2, ".xlsx":3, ".xls":3, ".parquet":4, ".json":5, ".txt":6}

def file_score(p):
    p_low = p.lower()
    score = min([ext_priority[e] for e in ext_priority if p_low.endswith(e)])
    return (score, -getsize(p))

candidates_sorted = sorted(candidates, key=file_score)
data_file = candidates_sorted[0]
print("Selected file:", data_file)

# ---- Step 2c (fixed): robust reader without the low_memory+python conflict ----
import pandas as pd

def read_any(path, nrows=None):
    p = path.lower()

    # CSV / TSV / TXT
    if p.endswith((".csv", ".csv.gz", ".tsv", ".txt")):
        # If it's clearly TSV, set sep="\t" and use default engine ("c")
        if p.endswith(".tsv"):
            return pd.read_csv(path, sep="\t", nrows=nrows)  # default engine OK
        # Otherwise, try automatic separator inference using the python engine,
        # but DO NOT pass low_memory (that triggers your error).
        try:
            return pd.read_csv(path, sep=None, engine="python", nrows=nrows)
        except Exception as e:
            print("[read_any] Fallback: trying default engine with comma sep due to:", e)
            return pd.read_csv(path, sep=",", nrows=nrows)

    # Excel
    if p.endswith((".xlsx", ".xls")):
        return pd.read_excel(path)

    # Parquet
    if p.endswith(".parquet"):
        return pd.read_parquet(path)

    # JSON (supports JSON lines or standard)
    if p.endswith(".json"):
        try:
            return pd.read_json(path, lines=True)
        except ValueError:
            return pd.read_json(path)

    raise ValueError(f"Unsupported file type: {path}")

# Use the loader
print("Selected file:", data_file)
df_raw = read_any(data_file)
print("Raw shape:", df_raw.shape)
df_raw.head()


Found files initially:
 - /kaggle/input/sales-data-for-economic-data-analysis/salesforcourse-4fe2kehu.csv
 - /kaggle/input/sales-data-for-economic-data-analysis/salesforcourse-4fe2kehu.xlsx

Files after unzip:
 - /kaggle/input/sales-data-for-economic-data-analysis/salesforcourse-4fe2kehu.csv
 - /kaggle/input/sales-data-for-economic-data-analysis/salesforcourse-4fe2kehu.xlsx
Selected file: /kaggle/input/sales-data-for-economic-data-analysis/salesforcourse-4fe2kehu.csv
Selected file: /kaggle/input/sales-data-for-economic-data-analysis/salesforcourse-4fe2kehu.csv
Raw shape: (34867, 16)


/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,index,Date,Year,Month,Customer Age,Customer Gender,Country,State,Product Category,Sub Category,Quantity,Unit Cost,Unit Price,Cost,Revenue,Column1
0,0,2/19/2016,2016.0,February,29.0,F,United States,Washington,Accessories,Tires and Tubes,1.0,80.00,109.000000,80.0,109.0,NaN
1,1,2/20/2016,2016.0,February,29.0,F,United States,Washington,Clothing,Gloves,2.0,24.50,28.500000,49.0,57.0,NaN
2,2,2/27/2016,2016.0,February,29.0,F,United States,Washington,Accessories,Tires and Tubes,3.0,3.67,5.000000,11.0,15.0,NaN
3,3,3/12/2016,2016.0,March,29.0,F,United States,Washington,Accessories,Tires and Tubes,2.0,87.50,116.500000,175.0,233.0,NaN
4,4,3/12/2016,2016.0,March,29.0,F,United States,Washington,Accessories,Tires and Tubes,3.0,35.00,41.666667,105.0,125.0,NaN


### Step 3. Normalize column names
Convert all column names to lowercase, snake_case, and remove special characters for consistency.


In [10]:
def normalize_cols(cols):
    out=[]
    for c in cols:
        c2= c.strip().lower()
        c2= re.sub(r"[\s/]+","_",c2)
        c2= re.sub(r"[^0-9a-zA-Z_]","",c2)
        out.append(c2)
    return out
df_raw.columns = normalize_cols(df_raw.columns)
df_raw.head()

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,index,date,year,month,customer_age,customer_gender,country,state,product_category,sub_category,quantity,unit_cost,unit_price,cost,revenue,column1
0,0,2/19/2016,2016.0,February,29.0,F,United States,Washington,Accessories,Tires and Tubes,1.0,80.00,109.000000,80.0,109.0,NaN
1,1,2/20/2016,2016.0,February,29.0,F,United States,Washington,Clothing,Gloves,2.0,24.50,28.500000,49.0,57.0,NaN
2,2,2/27/2016,2016.0,February,29.0,F,United States,Washington,Accessories,Tires and Tubes,3.0,3.67,5.000000,11.0,15.0,NaN
3,3,3/12/2016,2016.0,March,29.0,F,United States,Washington,Accessories,Tires and Tubes,2.0,87.50,116.500000,175.0,233.0,NaN
4,4,3/12/2016,2016.0,March,29.0,F,United States,Washington,Accessories,Tires and Tubes,3.0,35.00,41.666667,105.0,125.0,NaN


### Step 4. Identify key fields
We search for potential columns representing:
- **Revenue (sales)**  
- **Cost (COGS)**  
- **Quantity, price** (if totals are not given)  
- **Date**  


In [12]:
def find_one(patterns):
    for p in patterns:
        for c in df_raw.columns:
            if re.search(p,c,flags=re.I):
                return c
        return none

date_col = find_one([r"date"])
qty_col  = find_one([r"qty|quantity"])
rev_col  = find_one([r"revenue|sales|amount"])
cost_col = find_one([r"cost|cogs|expense"])

print("date column",date_col)
print("Quantity",qty_col)
print("Revenue",rev_col)
print("Cost",cost_col)

date column date
Quantity quantity
Revenue revenue
Cost unit_cost


### Step 5. Clean and convert data types
- Parse dates properly  
- Convert numeric fields to floats  
- Handle missing values  


In [13]:
df= df_raw.copy()

if date_col:
    df[date_col] = pd.to_datetime(df[date_col],errors="coerce")

for col in [rev_col, cost_col, qty_col]:
    if col and col in df.columns:
        df[col]= pd.to_numeric(df[col],errors="coerce")

### Step 6. Derive P&L metrics
We standardize the fields to:
- `Revenue`  
- `Cost`  
- `Profit = Revenue - Cost`  
- `Profit_Margin = Profit / Revenue`  


In [14]:
df["Revenue"]=df[rev_col]
df["Cost"]   =df[cost_col]
df["Profit"] =df["Revenue"] - df["Cost"]
df["Profit_Margin"]= df["Profit"] / df["Revenue"]

df[["Revenue","Cost","Profit","Profit_Margin"]].head()

,Revenue,Cost,Profit,Profit_Margin
0,109.0,80.00,29.00,0.266055
1,57.0,24.50,32.50,0.570175
2,15.0,3.67,11.33,0.755333
3,233.0,87.50,145.50,0.624464
4,125.0,35.00,90.00,0.720000


### Step 7. Add date intelligence
We create Year, Month, and Quarter columns to allow time-based slicing in Power BI.


In [15]:
df["Year"] = df[date_col].dt.year
df["Month"]= df[date_col].dt.month
df["Quarter"]  = "Q" + df[date_col].dt.quarter.astype(str)

df[["Year","Month","Quarter"]].head()

,Year,Month,Quarter
0,2016.0,2.0,Q1.0
1,2016.0,2.0,Q1.0
2,2016.0,2.0,Q1.0
3,2016.0,3.0,Q1.0
4,2016.0,3.0,Q1.0


### Step 8. Export cleaned dataset
We now save the cleaned dataset as `fact_sales.csv` for loading into Power BI.


In [16]:
out_path = "/kaggle/working/fact_sales.csv"
df.to_csv(out_path, index=False)
print("saved cleaned dataset to", out_path)

saved cleaned dataset to /kaggle/working/fact_sales.csv
